In [3]:
from __future__ import print_function, division
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

In [4]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [3]:
if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=20000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
__________

/Users/jonnyyu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.690941, acc.: 45.31%] [G loss: 0.671573]
1 [D loss: 0.689891, acc.: 50.00%] [G loss: 0.670500]
2 [D loss: 0.685899, acc.: 50.00%] [G loss: 0.667582]
3 [D loss: 0.680312, acc.: 50.00%] [G loss: 0.667224]
4 [D loss: 0.673560, acc.: 50.00%] [G loss: 0.669021]
5 [D loss: 0.669012, acc.: 50.00%] [G loss: 0.661952]
6 [D loss: 0.663094, acc.: 50.00%] [G loss: 0.662177]
7 [D loss: 0.654142, acc.: 50.00%] [G loss: 0.661013]
8 [D loss: 0.640273, acc.: 50.00%] [G loss: 0.658316]
9 [D loss: 0.627492, acc.: 50.00%] [G loss: 0.658509]
10 [D loss: 0.615815, acc.: 51.56%] [G loss: 0.663364]
11 [D loss: 0.603349, acc.: 50.00%] [G loss: 0.660880]
12 [D loss: 0.577314, acc.: 54.69%] [G loss: 0.662043]
13 [D loss: 0.557115, acc.: 53.12%] [G loss: 0.661996]
14 [D loss: 0.522976, acc.: 64.06%] [G loss: 0.675262]
15 [D loss: 0.511180, acc.: 62.50%] [G loss: 0.666008]
16 [D loss: 0.477279, acc.: 67.19%] [G loss: 0.697800]
17 [D loss: 0.457489, acc.: 71.88%] [G loss: 0.733581]
18 [D loss: 0.405466

149 [D loss: 0.288533, acc.: 87.50%] [G loss: 6.758997]
150 [D loss: 0.492703, acc.: 78.12%] [G loss: 5.745538]
151 [D loss: 0.269411, acc.: 87.50%] [G loss: 5.809718]
152 [D loss: 0.303071, acc.: 87.50%] [G loss: 4.338922]
153 [D loss: 0.367105, acc.: 87.50%] [G loss: 5.329690]
154 [D loss: 0.351547, acc.: 84.38%] [G loss: 4.546536]
155 [D loss: 0.517107, acc.: 81.25%] [G loss: 4.763511]
156 [D loss: 0.305570, acc.: 89.06%] [G loss: 4.616237]
157 [D loss: 0.293831, acc.: 87.50%] [G loss: 5.325326]
158 [D loss: 0.715323, acc.: 60.94%] [G loss: 4.380009]
159 [D loss: 0.311180, acc.: 89.06%] [G loss: 4.229812]
160 [D loss: 0.313420, acc.: 87.50%] [G loss: 4.865912]
161 [D loss: 0.284635, acc.: 85.94%] [G loss: 4.537816]
162 [D loss: 0.318304, acc.: 85.94%] [G loss: 6.236296]
163 [D loss: 0.344910, acc.: 85.94%] [G loss: 4.695121]
164 [D loss: 0.376324, acc.: 84.38%] [G loss: 4.665172]
165 [D loss: 0.400066, acc.: 79.69%] [G loss: 5.344653]
166 [D loss: 0.309443, acc.: 82.81%] [G loss: 6.

297 [D loss: 0.514907, acc.: 82.81%] [G loss: 3.087268]
298 [D loss: 0.492419, acc.: 73.44%] [G loss: 2.611886]
299 [D loss: 0.401777, acc.: 76.56%] [G loss: 3.277180]
300 [D loss: 0.461016, acc.: 82.81%] [G loss: 2.449581]
301 [D loss: 0.357535, acc.: 87.50%] [G loss: 3.000450]
302 [D loss: 0.455299, acc.: 76.56%] [G loss: 4.118245]
303 [D loss: 0.418816, acc.: 81.25%] [G loss: 3.014785]
304 [D loss: 0.490387, acc.: 81.25%] [G loss: 2.815234]
305 [D loss: 0.393067, acc.: 85.94%] [G loss: 3.112566]
306 [D loss: 0.381756, acc.: 85.94%] [G loss: 2.178354]
307 [D loss: 0.274462, acc.: 90.62%] [G loss: 2.851619]
308 [D loss: 0.462379, acc.: 79.69%] [G loss: 3.321864]
309 [D loss: 0.509100, acc.: 75.00%] [G loss: 2.782908]
310 [D loss: 0.391724, acc.: 84.38%] [G loss: 2.725190]
311 [D loss: 0.364292, acc.: 84.38%] [G loss: 3.001440]
312 [D loss: 0.482693, acc.: 81.25%] [G loss: 2.377409]
313 [D loss: 0.483405, acc.: 76.56%] [G loss: 2.174090]
314 [D loss: 0.393535, acc.: 85.94%] [G loss: 2.

...

19343 [D loss: 0.628881, acc.: 65.62%] [G loss: 0.788260]
19344 [D loss: 0.657650, acc.: 57.81%] [G loss: 0.793628]
19345 [D loss: 0.670408, acc.: 64.06%] [G loss: 0.766382]
19346 [D loss: 0.698067, acc.: 50.00%] [G loss: 0.780460]
19347 [D loss: 0.707282, acc.: 53.12%] [G loss: 0.765413]
19348 [D loss: 0.748587, acc.: 48.44%] [G loss: 0.806136]
19349 [D loss: 0.663309, acc.: 57.81%] [G loss: 0.788945]
19350 [D loss: 0.657433, acc.: 62.50%] [G loss: 0.808483]
19351 [D loss: 0.667734, acc.: 56.25%] [G loss: 0.790650]
19352 [D loss: 0.663583, acc.: 57.81%] [G loss: 0.779554]
19353 [D loss: 0.672507, acc.: 64.06%] [G loss: 0.835630]
19354 [D loss: 0.709975, acc.: 48.44%] [G loss: 0.732455]
19355 [D loss: 0.703482, acc.: 56.25%] [G loss: 0.790258]
19356 [D loss: 0.688169, acc.: 53.12%] [G loss: 0.812401]
19357 [D loss: 0.654249, acc.: 60.94%] [G loss: 0.774886]
19358 [D loss: 0.695000, acc.: 56.25%] [G loss: 0.767315]
19359 [D loss: 0.667359, acc.: 62.50%] [G loss: 0.776507]
19360 [D loss:

19485 [D loss: 0.650614, acc.: 57.81%] [G loss: 0.764884]
19486 [D loss: 0.640315, acc.: 59.38%] [G loss: 0.765006]
19487 [D loss: 0.654728, acc.: 56.25%] [G loss: 0.730061]
19488 [D loss: 0.670580, acc.: 59.38%] [G loss: 0.737570]
19489 [D loss: 0.663826, acc.: 59.38%] [G loss: 0.759011]
19490 [D loss: 0.629230, acc.: 59.38%] [G loss: 0.746515]
19491 [D loss: 0.635041, acc.: 65.62%] [G loss: 0.711963]
19492 [D loss: 0.674295, acc.: 53.12%] [G loss: 0.830666]
19493 [D loss: 0.633066, acc.: 60.94%] [G loss: 0.806132]
19494 [D loss: 0.617887, acc.: 62.50%] [G loss: 0.802253]
19495 [D loss: 0.713121, acc.: 50.00%] [G loss: 0.846083]
19496 [D loss: 0.693947, acc.: 54.69%] [G loss: 0.775348]
19497 [D loss: 0.680555, acc.: 54.69%] [G loss: 0.757176]
19498 [D loss: 0.697127, acc.: 48.44%] [G loss: 0.800200]
19499 [D loss: 0.668027, acc.: 59.38%] [G loss: 0.774325]
19500 [D loss: 0.638803, acc.: 64.06%] [G loss: 0.807602]
19501 [D loss: 0.709634, acc.: 51.56%] [G loss: 0.819539]
19502 [D loss:

19628 [D loss: 0.696836, acc.: 51.56%] [G loss: 0.803972]
19629 [D loss: 0.675313, acc.: 56.25%] [G loss: 0.794240]
19630 [D loss: 0.672835, acc.: 54.69%] [G loss: 0.735282]
19631 [D loss: 0.669700, acc.: 60.94%] [G loss: 0.756152]
19632 [D loss: 0.694510, acc.: 48.44%] [G loss: 0.796494]
19633 [D loss: 0.699239, acc.: 59.38%] [G loss: 0.762837]
19634 [D loss: 0.654454, acc.: 60.94%] [G loss: 0.804460]
19635 [D loss: 0.665433, acc.: 56.25%] [G loss: 0.809228]
19636 [D loss: 0.660102, acc.: 57.81%] [G loss: 0.796704]
19637 [D loss: 0.700379, acc.: 50.00%] [G loss: 0.745348]
19638 [D loss: 0.676375, acc.: 62.50%] [G loss: 0.768851]
19639 [D loss: 0.702164, acc.: 48.44%] [G loss: 0.770760]
19640 [D loss: 0.652114, acc.: 59.38%] [G loss: 0.775801]
19641 [D loss: 0.657516, acc.: 53.12%] [G loss: 0.788077]
19642 [D loss: 0.666675, acc.: 62.50%] [G loss: 0.761396]
19643 [D loss: 0.694331, acc.: 53.12%] [G loss: 0.763706]
19644 [D loss: 0.697846, acc.: 56.25%] [G loss: 0.779660]
19645 [D loss:

19771 [D loss: 0.670939, acc.: 59.38%] [G loss: 0.774200]
19772 [D loss: 0.633289, acc.: 73.44%] [G loss: 0.761843]
19773 [D loss: 0.658431, acc.: 65.62%] [G loss: 0.831559]
19774 [D loss: 0.692220, acc.: 50.00%] [G loss: 0.754744]
19775 [D loss: 0.699294, acc.: 53.12%] [G loss: 0.761021]
19776 [D loss: 0.714583, acc.: 48.44%] [G loss: 0.747301]
19777 [D loss: 0.664293, acc.: 53.12%] [G loss: 0.758605]
19778 [D loss: 0.662957, acc.: 64.06%] [G loss: 0.782274]
19779 [D loss: 0.658385, acc.: 60.94%] [G loss: 0.769500]
19780 [D loss: 0.699676, acc.: 53.12%] [G loss: 0.798364]
19781 [D loss: 0.605028, acc.: 67.19%] [G loss: 0.733003]
19782 [D loss: 0.671075, acc.: 64.06%] [G loss: 0.770899]
19783 [D loss: 0.747085, acc.: 45.31%] [G loss: 0.778663]
19784 [D loss: 0.666606, acc.: 54.69%] [G loss: 0.778533]
19785 [D loss: 0.661022, acc.: 53.12%] [G loss: 0.806823]
19786 [D loss: 0.681305, acc.: 54.69%] [G loss: 0.789765]
19787 [D loss: 0.663796, acc.: 54.69%] [G loss: 0.760891]
19788 [D loss:

19914 [D loss: 0.699919, acc.: 45.31%] [G loss: 0.700402]
19915 [D loss: 0.656186, acc.: 62.50%] [G loss: 0.746007]
19916 [D loss: 0.707502, acc.: 48.44%] [G loss: 0.773115]
19917 [D loss: 0.632064, acc.: 60.94%] [G loss: 0.722352]
19918 [D loss: 0.641449, acc.: 64.06%] [G loss: 0.739930]
19919 [D loss: 0.665399, acc.: 51.56%] [G loss: 0.769065]
19920 [D loss: 0.652656, acc.: 59.38%] [G loss: 0.787648]
19921 [D loss: 0.662275, acc.: 51.56%] [G loss: 0.773054]
19922 [D loss: 0.678347, acc.: 54.69%] [G loss: 0.819473]
19923 [D loss: 0.669101, acc.: 53.12%] [G loss: 0.793680]
19924 [D loss: 0.706566, acc.: 53.12%] [G loss: 0.769198]
19925 [D loss: 0.659060, acc.: 53.12%] [G loss: 0.827584]
19926 [D loss: 0.674385, acc.: 56.25%] [G loss: 0.724403]
19927 [D loss: 0.666780, acc.: 62.50%] [G loss: 0.763192]
19928 [D loss: 0.666529, acc.: 60.94%] [G loss: 0.769910]
19929 [D loss: 0.658482, acc.: 68.75%] [G loss: 0.784573]
19930 [D loss: 0.662473, acc.: 56.25%] [G loss: 0.782358]
19931 [D loss: